# 🚦 YOLO Detection Server V3
### Multi-Camera FLV Stream + HTTP POST Results
### Vietnamese License Plate Recognition (YOLOv8)

In [ ]:
# ═══════════════════════════════════════════════════════════
# Cell 1: CONFIGURATION - EDIT THIS!
# ═══════════════════════════════════════════════════════════

BACKEND_URL = 'https://plugin-border-bare-proportion.trycloudflare.com'
NMS_URL = 'https://initiatives-booth-wright-hip.trycloudflare.com'

TRACK_LINE_Y = 50
CONFIDENCE = 0.5
FPS_LIMIT = 10

print(f"📡 Backend: {BACKEND_URL}")
print(f"📺 NMS: {NMS_URL}")

In [ ]:
# Cell 2: Install Dependencies + Download Vietnamese LP Models
!pip install ultralytics opencv-python-headless requests --quiet
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt

# Download Vietnamese License Plate models from GitHub (MagicXuanTung)
import os
if not os.path.exists('license_plate_detector.pt'):
    !wget -q https://github.com/MagicXuanTung/Yolov8-Detect-Vietnamese-license-plates-and-characters/raw/main/license_plate_detector.pt
    print('✅ Downloaded license_plate_detector.pt')
if not os.path.exists('character_detector.pt'):
    !wget -q https://github.com/MagicXuanTung/Yolov8-Detect-Vietnamese-license-plates-and-characters/raw/main/character_detector.pt
    print('✅ Downloaded character_detector.pt')

print('✅ All dependencies ready!')

In [ ]:
# Cell 3: Load Models
import warnings
warnings.filterwarnings('ignore')
import os, torch
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'🚀 Device: {device}')

vehicle_model = traffic_light_model = lp_detector = char_detector = None

# Vehicle Detection
try:
    vehicle_model = YOLO('yolo11m.pt').to(device)
    print('✅ Vehicle')
except Exception as e: print(f'⚠️ Vehicle: {e}')

# Traffic Light Detection
try:
    p = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/mhiot-dentinhieu-best-new.pt'
    if os.path.exists(p):
        traffic_light_model = YOLO(p).to(device)
        print('✅ Traffic Light')
except Exception as e: print(f'⚠️ TL: {e}')

# Vietnamese License Plate Detection
try:
    lp_detector = YOLO('license_plate_detector.pt').to(device)
    print('✅ LP Detector (YOLOv8)')
except Exception as e: print(f'⚠️ LP Detector: {e}')

# Vietnamese Character Detection
try:
    char_detector = YOLO('character_detector.pt').to(device)
    print('✅ Character Detector (YOLOv8)')
except Exception as e: print(f'⚠️ Char Detector: {e}')

print(f"\n📊 V:{'✅' if vehicle_model else '❌'} TL:{'✅' if traffic_light_model else '❌'} LP:{'✅' if lp_detector else '❌'} OCR:{'✅' if char_detector else '❌'}")

In [ ]:
# Cell 4: Detection Functions
import cv2, numpy as np, time, requests, threading, re
from datetime import datetime

VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']
camera_trackers = {}

def log(msg): print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

def read_plate_characters(plate_img):
    """Read characters from license plate image using YOLOv8 character detector"""
    if char_detector is None or plate_img.size == 0:
        return 'unknown'
    try:
        results = char_detector(plate_img, verbose=False)
        chars = []
        for r in results:
            for b in r.boxes:
                if float(b.conf[0]) < 0.3: continue
                x1 = int(b.xyxy[0][0])
                cls_id = int(b.cls[0])
                char_val = char_detector.names[cls_id]
                chars.append((x1, char_val))
        
        if len(chars) < 5: return 'unknown'
        
        # Sort characters by x-coordinate (left to right)
        chars.sort(key=lambda x: x[0])
        text = ''.join([c[1] for c in chars])
        
        # Basic cleanup
        text = text.upper().replace(' ', '').replace('-', '')
        return text
    except: return 'unknown'

def detect_license_plate(frame, vehicle_bbox):
    """Detect and read Vietnamese license plate within vehicle bounding box"""
    if lp_detector is None: return None
    
    x1, y1, x2, y2 = vehicle_bbox
    vehicle_crop = frame[y1:y2, x1:x2]
    if vehicle_crop.size == 0: return None
    
    try:
        results = lp_detector(vehicle_crop, verbose=False)
        # Find best plate detection
        best_plate = None
        max_conf = 0
        
        for r in results:
            for b in r.boxes:
                conf = float(b.conf[0])
                if conf > max_conf and conf > 0.3:
                    max_conf = conf
                    best_plate = b
        
        if best_plate:
            px1, py1, px2, py2 = map(int, best_plate.xyxy[0])
            plate_crop = vehicle_crop[py1:py2, px1:px2]
            
            # Read characters
            text = read_plate_characters(plate_crop)
            
            if text != 'unknown':
                return {
                    'text': text,
                    'confidence': max_conf,
                    'bbox_pixels': [x1+px1, y1+py1, x1+px2, y1+py2]
                }
    except: pass
    return None

def detect_frame(frame, camera_id):
    h, w = frame.shape[:2]
    result = {
        'camera_id': camera_id,
        'created_at': int(time.time() * 1000),
        'image_dimensions': {'width': w, 'height': h},
        'track_line_y': TRACK_LINE_Y
    }
    
    if camera_id not in camera_trackers:
        camera_trackers[camera_id] = {
            'tracks': {}, 'counted': {},
            'counts_up': {v:0 for v in VEHICLE_CLASSES},
            'counts_down': {v:0 for v in VEHICLE_CLASSES},
            'total_up': 0, 'total_down': 0
        }
    tr = camera_trackers[camera_id]
    
    # Vehicle Detection
    if vehicle_model:
        t0 = time.time()
        dets, tracks, vcounts, crossings = [], {}, {v:0 for v in VEHICLE_CLASSES}, []
        ly = int(h * TRACK_LINE_Y / 100)
        
        for r in vehicle_model.track(frame, persist=True, verbose=False):
            for b in r.boxes:
                cls = vehicle_model.names[int(b.cls[0])]
                if cls not in VEHICLE_CLASSES or float(b.conf[0]) < CONFIDENCE: continue
                x1,y1,x2,y2 = map(int, b.xyxy[0])
                cx, cy = (x1+x2)//2, (y1+y2)//2
                det = {
                    'class': cls, 'type': 'vehicle', 'confidence': float(b.conf[0]),
                    'bbox': {'x1': x1/w, 'y1': y1/h, 'x2': x2/w, 'y2': y2/h, 'width': (x2-x1)/w, 'height': (y2-y1)/h},
                    'bbox_pixels': [x1, y1, x2, y2]
                }
                if hasattr(b,'id') and b.id is not None:
                    tid = int(b.id[0]); det['id'] = tid
                    tracks[tid] = {'pos': (cx,cy), 'time': result['created_at'], 'class': cls}
                
                # License Plate Detection
                lp = detect_license_plate(frame, [x1, y1, x2, y2])
                if lp: det['license_plate'] = lp
                
                dets.append(det); vcounts[cls] += 1
        
        # Counting logic
        for tid, info in tracks.items():
            if tid not in tr['tracks']: tr['tracks'][tid] = []
            if tr['tracks'][tid]:
                py, cy = tr['tracks'][tid][-1]['pos'][1], info['pos'][1]
                d = 1 if py <= ly < cy else (-1 if py >= ly > cy else 0)
                if d and f"{tid}_{d}" not in tr['counted']:
                    tr['counted'][f"{tid}_{d}"] = True
                    if d == 1: tr['counts_down'][info['class']] += 1; tr['total_down'] += 1
                    else: tr['counts_up'][info['class']] += 1; tr['total_up'] += 1
                    crossings.append({'id': tid, 'direction': d})
            tr['tracks'][tid].append({'pos': info['pos'], 'time': info['time'], 'class': info['class']})
            tr['tracks'][tid] = tr['tracks'][tid][-30:]
        
        result['vehicle'] = {
            'detections': dets, 'inference_time': (time.time()-t0)*1000,
            'vehicle_count': {
                'total_up': tr['total_up'], 'total_down': tr['total_down'],
                'by_type_up': tr['counts_up'].copy(), 'by_type_down': tr['counts_down'].copy(),
                'current': vcounts
            },
            'new_crossings': crossings,
            'tracks': [{'id': tid, 'positions': [{'x': p['pos'][0], 'y': p['pos'][1], 'time': p['time']} for p in tdata], 'class': tdata[-1]['class']} for tid, tdata in tr['tracks'].items() if tdata and tid in tracks]
        }
    
    # Traffic Light Detection
    if traffic_light_model:
        t0 = time.time(); tl_dets = []; status = None; mx = 0
        for r in traffic_light_model(frame, verbose=False):
            for b in r.boxes:
                cf = float(b.conf[0])
                if cf < 0.4: continue
                tx1,ty1,tx2,ty2 = map(int, b.xyxy[0])
                cn = traffic_light_model.names[int(b.cls[0])]
                tl_dets.append({'class': cn, 'confidence': cf, 'bbox': {'x1':tx1/w, 'y1':ty1/h, 'x2':tx2/w, 'y2':ty2/h, 'width':(tx2-tx1)/w, 'height':(ty2-ty1)/h}, 'bbox_pixels': [tx1, ty1, tx2, ty2]})
                if cf > mx: mx, status = cf, cn
        result['traffic_light'] = {'detections': tl_dets, 'traffic_status': status, 'inference_time': (time.time()-t0)*1000}
    
    return result

print('✅ Detection functions ready')

In [ ]:
# Cell 5: Multi-Camera Stream Processor
import cv2, time, requests, threading
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

camera_stats = {}
session = requests.Session()
retry = Retry(total=3, backoff_factor=0.5, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retry))
session.mount('https://', HTTPAdapter(max_retries=retry))

def fetch_cameras():
    try:
        resp = session.get(f"{BACKEND_URL}/api/camera/all", timeout=10)
        if resp.status_code == 200:
            data = resp.json()
            cameras = data.get('metadata', data) if isinstance(data, dict) else data
            return [cam['_id'] for cam in cameras if isinstance(cam, dict) and cam.get('_id')]
    except Exception as e: log(f'❌ Fetch cameras error: {e}')
    return []

def post_result(camera_id, result):
    try:
        resp = session.post(f"{BACKEND_URL}/api/detection/{camera_id}", json=result, timeout=5)
        if resp.status_code == 200: camera_stats[camera_id]['sent'] += 1
    except: camera_stats[camera_id]['errors'] += 1

def process_camera(camera_id):
    flv_url = f"{NMS_URL}/live/{camera_id}.flv"
    camera_stats[camera_id] = {'frames': 0, 'sent': 0, 'errors': 0}
    log(f'🎥 [{camera_id[-8:]}] Connecting...')
    frame_interval = 1.0 / FPS_LIMIT
    last_detect_time = 0
    
    while True:
        try:
            cap = cv2.VideoCapture(flv_url)
            if not cap.isOpened(): time.sleep(5); continue
            log(f'✅ [{camera_id[-8:]}] Connected')
            
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret: break
                camera_stats[camera_id]['frames'] += 1
                now = time.time()
                if now - last_detect_time >= frame_interval:
                    last_detect_time = now
                    result = detect_frame(frame, camera_id)
                    post_result(camera_id, result)
            cap.release()
        except Exception as e:
            log(f'❌ [{camera_id[-8:]}] {e}')
            time.sleep(5)

def print_stats():
    while True:
        time.sleep(60)
        total_f = sum(s['frames'] for s in camera_stats.values())
        total_s = sum(s['sent'] for s in camera_stats.values())
        total_e = sum(s['errors'] for s in camera_stats.values())
        log(f'📊 Total: {total_f} frames | {total_s} sent | {total_e} errors')

log('🔍 Fetching cameras...')
cameras = fetch_cameras()
if not cameras:
    log('❌ No cameras found!')
else:
    log(f'✅ Found {len(cameras)} cameras')
    threading.Thread(target=print_stats, daemon=True).start()
    threads = []
    for cam_id in cameras:
        t = threading.Thread(target=process_camera, args=(cam_id,), daemon=True)
        t.start(); threads.append(t)
    log('🚀 Started!')
    for t in threads: t.join()